In [122]:
import nltk
import random
### import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu

import random
random.seed(42)

In [3]:
from bert_score import score

In [108]:
with open('./training_data/AllVideoDescriptions.txt') as f:
    lines = f.readlines()

In [109]:
with open('./test_split.txt','r') as f:
    test_l = f.readlines()
test_l = [l.replace('\n','') for l in test_l]

In [110]:
scripts = {}
for l in lines:
    id = l.split(' ')[0]
    script = ' '.join(l.split(' ')[1:])
    if id in scripts:
      scripts[id].append(script)
    else:
      scripts[id] = [script]
scripts = {k:list(set(v)) for k,v in scripts.items()}

In [148]:
model_path = './model_att_general_2022-06-09 /'
method ='beam'

In [149]:
with open(f'{model_path}/test_output_{method}.txt','r') as f:
    l = f.readlines()

In [123]:
# blues_all= []
# meteors_all= []
# bert =[]
# result_all=[]
# for gen in l:
#     results= {}
#     label_id = gen.split(',')[0]
#     label = [i.replace('\n','').split(' ') for i in scripts[label_id]]
#     label_sentence= [[l.replace('\n','') for l in scripts[label_id]]]
#     gen_sentence =  [gen.split(',')[1].replace('\n','')]
#     gen = gen.split(',')[1].replace('\n','').split(' ')
#     gen = [g for g in gen if g!=' ' and g!='']
#     blue =  sentence_bleu(
#     label, gen,weights=[(1., 0), (0.5, 0.5), (0.333, 0.333, 0.334), (0.25, 0.25, 0.25, 0.25)])
#     blues_all.append(blue)
#     meteors= meteor_score(label, gen)
#     meteors_all.append(meteors)
# #    _,_,berts = score(gen_sentence, label_sentence, lang="en", verbose=True)
# #    bert.append(berts)
#     results['video_id'] = label_id
#     results['label'] = [" ".join(l) for l in label]
#     results['gen'] = " ".join(gen)
#     results['meteor'] = meteors
#     results['bleu'] = blue
#     results['bert_score'] = berts
#     result_all.append(results)

/home/jinhwak/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jinhwak/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jinhwak/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

In [150]:
blues_all= []
meteors_all= []
with open(f'./{model_path}/test_eval_{method}.txt','w') as f:
    print(f'./{model_path}/test_eval_{method}.txt')
    for gen in l:
        results= {}
        label_id = gen.split(',')[0]
        label = [i.replace('\n','').split(' ') for i in scripts[label_id]]
        gen = gen.split(',')[1].replace('\n','').split(' ')
        gen = [g for g in gen if g!=' ' and g!='']
        blue =  sentence_bleu(
        label, gen,weights=[(1., 0), (0.5, 0.5), (0.333, 0.333, 0.334), (0.25, 0.25, 0.25, 0.25)])
        blues_all.append(blue)
        meteors= nltk.translate.meteor_score.meteor_score(
         label, gen)
        meteors_all.append(meteors)
        results['video_id'] = label_id
        results['label'] = [" ".join(l) for l in label]
        results['gen'] = " ".join(gen)
        results['meteor'] = meteors
        results['bleu'] = blue
        f.write(json.dumps(results))
        f.write("\n")
    f.write(f"Final BLEU: {np.mean(np.array(blues_all),axis=0)} METEOR: {np.mean(meteors_all)}")
    print(f"Final BLEU: {np.mean(np.array(blues_all),axis=0)} METEOR: {np.mean(meteors_all)}")
        

././model_att_general_2022-06-09 //test_eval_beam.txt
Final BLEU: [0.70267764 0.48326246 0.333825   0.15450715] METEOR: 0.5145667519365587
